In [2]:
import pandas as pd
import os

print(os.getcwd())
os.chdir('/home/lyq/DataSet/FakeNews/weibo_dataset') # 替换为自己的weibo数据集所在文件夹
os.getcwd()


/home/lyq/PycharmProjects/QwenVLRationaleGenerate/data/weibo


'/home/lyq/DataSet/FakeNews/weibo_dataset'

In [3]:

from urllib.parse import urlparse


def extract_filename_from_url(url):
    # 解析URL
    parsed_url = urlparse(url)
    # 获取路径
    path = parsed_url.path
    # 分割路径以获取最后一部分，即文件名
    filename = path.split('/')[-1]
    # 如果文件名为空，则可能没有文件名，返回None
    if not filename:
        return None
    return filename

def is_valid_url(url):
    try:
        result = urlparse(url)
        # 检查是否有scheme和netloc，这是构成有效URL的基本要素
        return all([result.scheme, result.netloc])
    except:
        return False

def read_weibo_data_file(file_path):
    split,auth = file_path.split('/')[-1].split('.')[0].split('_')
    label = 'fake' if auth == 'rumor' else 'real'
    result = {
        'id':[],
        'text':[],
        'release_source':[],
        'image_file_names':[],
        'split':[],
        'label':[]
    }
    with open(file_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
        for i in range(0,len(lines),3):
            result['id'].append(str(lines[i].split('|')[0]))
            result['text'].append(lines[i+2].strip())
            image_file_names = [extract_filename_from_url(url) for url in lines[i+1].strip().split('|') if is_valid_url(url)]
            result['image_file_names'].append(image_file_names)
            result['release_source'].append(lines[i].strip().split('|')[-1])

    result['split'] = [split for _ in range(len(result['id']))]
    result['label'] = [label for _ in range(len(result['id']))]
    return result

df = pd.concat([ pd.DataFrame(read_weibo_data_file(f'tweets/{file}')) for file in os.listdir('tweets') if file.endswith('.txt')],ignore_index=True)

df

,id,text,release_source,image_file_names,split,label
0,3511947309647762,震惊，转发求证：【想都不敢想 ，在美国一桶金龙鱼食用油只要8元人民币】 一桶食用油相当于中国...,微博 weibo.com,[a71ac854gw1dytin2zmk9j.jpg],train,fake
1,3576100079039606,【法院无底线】湖南长沙一位小朋友上学路上捡到3万元，原地不动等失主，结果被人冒领。不知情的孩...,iPhone客户端,[3ba161e7jw1e4i6j4ep61j211n0r4juz.jpg],train,fake
2,3899073935617462,"转发:我校需要小孩的衣服,新旧不限!西藏阿里地区是世界海拔最高的地区请问周围有没有四到十岁孩...",vivo X5Pro,[006ajvCngw1ex4jjl2pwlj30c8096wf7.jpg],train,fake
3,3584521306131914,"立刻检查一下你家里的牙膏，如果是黑色条马上扔掉！ 大家买膏请留心,买牙膏时注意牙膏管反面 底...",Weico.Android,[6112c76ajw1e591p39mpej20hw094mxh.jpg],train,fake
4,3553661986467439,快快转发，急急急！中央电视台《焦点访谈》已经播出，可口可乐承认旗下(果粒橙)含有美国禁用农药...,三星Galaxy Note II,[a5404163jw1e2iljwbrztj.jpg],train,fake
...,...,...,...,...,...,...
9522,3896157270699471,【空城记：刘小东与鄂尔多斯】#正午故事#鄂尔多斯是个比较敏感的城市。其实政府也比想象的聪明，...,微博 weibo.com,"[005EHRy1jw1ewv1ay6i52j30rs0ij10c.jpg, 005EHRy...",test,real
9523,3895644894378737,【日本人是怎么拍黑帮电影的？】Tim Gallo出生于俄国，17岁后迁往日本生活。通过用镜头...,微博 weibo.com,"[005EHRy1jw1ewtlhdbkqbj30jg0czjsn.jpg, 005EHRy...",test,real
9524,3893257346088858,#跟着面叔看世界#第一站，日本东京筑地市场。你现在到哪儿了？不会还堵在路上吧？一起来晒晒旅途...,null,"[005EHRy1jw1ewlzj5ottxj30zk0qojxy.jpg, 005EHRy...",test,real
9525,3892828281041275,【现场图·广西柳城县连续发生多起爆炸】据@南宁电视台 ：网友称，现场听到几声巨响，车站，市场...,微博 weibo.com,"[61e04755jw1ewkm8o7fyyj20c80gb0u2.jpg, 61e0475...",test,real


In [4]:

def get_available_image_dict(image_dir):
    return {
        file_name:f'{image_dir}/{file_name}' for file_name in os.listdir(image_dir)
    }

available_image_dict = get_available_image_dict('rumor_images')
available_image_dict.update(get_available_image_dict('nonrumor_images'))
def filter_available_image(image_file_name_list):
    return [available_image_dict[image_file] for image_file in image_file_name_list if image_file in available_image_dict.keys()]

df['available_image_paths'] = df['image_file_names'].apply(filter_available_image)


df

,id,text,release_source,image_file_names,split,label,available_image_paths
0,3511947309647762,震惊，转发求证：【想都不敢想 ，在美国一桶金龙鱼食用油只要8元人民币】 一桶食用油相当于中国...,微博 weibo.com,[a71ac854gw1dytin2zmk9j.jpg],train,fake,[rumor_images/a71ac854gw1dytin2zmk9j.jpg]
1,3576100079039606,【法院无底线】湖南长沙一位小朋友上学路上捡到3万元，原地不动等失主，结果被人冒领。不知情的孩...,iPhone客户端,[3ba161e7jw1e4i6j4ep61j211n0r4juz.jpg],train,fake,[rumor_images/3ba161e7jw1e4i6j4ep61j211n0r4juz...
2,3899073935617462,"转发:我校需要小孩的衣服,新旧不限!西藏阿里地区是世界海拔最高的地区请问周围有没有四到十岁孩...",vivo X5Pro,[006ajvCngw1ex4jjl2pwlj30c8096wf7.jpg],train,fake,[rumor_images/006ajvCngw1ex4jjl2pwlj30c8096wf7...
3,3584521306131914,"立刻检查一下你家里的牙膏，如果是黑色条马上扔掉！ 大家买膏请留心,买牙膏时注意牙膏管反面 底...",Weico.Android,[6112c76ajw1e591p39mpej20hw094mxh.jpg],train,fake,[rumor_images/6112c76ajw1e591p39mpej20hw094mxh...
4,3553661986467439,快快转发，急急急！中央电视台《焦点访谈》已经播出，可口可乐承认旗下(果粒橙)含有美国禁用农药...,三星Galaxy Note II,[a5404163jw1e2iljwbrztj.jpg],train,fake,[rumor_images/a5404163jw1e2iljwbrztj.jpg]
...,...,...,...,...,...,...,...
9522,3896157270699471,【空城记：刘小东与鄂尔多斯】#正午故事#鄂尔多斯是个比较敏感的城市。其实政府也比想象的聪明，...,微博 weibo.com,"[005EHRy1jw1ewv1ay6i52j30rs0ij10c.jpg, 005EHRy...",test,real,[nonrumor_images/005EHRy1jw1ewv1azxwh1j30rs0ll...
9523,3895644894378737,【日本人是怎么拍黑帮电影的？】Tim Gallo出生于俄国，17岁后迁往日本生活。通过用镜头...,微博 weibo.com,"[005EHRy1jw1ewtlhdbkqbj30jg0czjsn.jpg, 005EHRy...",test,real,[nonrumor_images/005EHRy1jw1ewtlhknf30j30jg0cz...
9524,3893257346088858,#跟着面叔看世界#第一站，日本东京筑地市场。你现在到哪儿了？不会还堵在路上吧？一起来晒晒旅途...,null,"[005EHRy1jw1ewlzj5ottxj30zk0qojxy.jpg, 005EHRy...",test,real,[nonrumor_images/005EHRy1jw1ewlzj6oenaj30qo0zk...
9525,3892828281041275,【现场图·广西柳城县连续发生多起爆炸】据@南宁电视台 ：网友称，现场听到几声巨响，车站，市场...,微博 weibo.com,"[61e04755jw1ewkm8o7fyyj20c80gb0u2.jpg, 61e0475...",test,real,[nonrumor_images/61e04755jw1ewkm8o7fyyj20c80gb...


In [5]:
print(f'before filter: {df.shape[0]} real {(df["label"]=="real").sum()} fake {(df["label"]=="fake").sum()}')
df = df[(df['available_image_paths'].apply(len) > 0 ) & (df['text'] is not None)]
df = df.drop(columns=['image_file_names'])
df['release_source'] = df['release_source'].apply(lambda x:'未知' if x == 'null' else x)
print(f'before filter: {df.shape[0]} real {(df["label"]=="real").sum()} fake {(df["label"]=="fake").sum()}')

before filter: 9527 real 4779 fake 4748
before filter: 7961 real 3642 fake 4319


In [5]:



df.to_csv('weibo.csv',index=False)